##⚡Importação das Bibliotecas

O clássico "setup do projeto", onde a gente importa todos os pacotes que vai usar mais pra frente.

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import shutil
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
import matplotlib.patches as mpatches

##⚠️ Observação sobre o uso do dataset

Existem duas formas de acessar o dataset:

✅ Via opendatasets: Ideal para Google Colab. Permite baixar diretamente do Kaggle com od.download(url) — é necessário fornecer suas credenciais do Kaggle (usuário + API key).

🔒 Via caminho local (ex: /kaggle/input/...): Funciona apenas dentro dos notebooks do Kaggle, pois os dados já estão incluídos no ambiente e não precisam ser baixados manualmente.

➕ Use opendatasets no Colab ou Jupyter local.
➖ Use caminho local apenas se estiver rodando o notebook dentro da plataforma Kaggle.

Nesse exemplo, os autores usaram o caminho direto do Kaggle, mas quando NÓS fizermos isso no colab, precisamos importar com o opendatasets✌

In [ ]:
base_dir  = '/kaggle/input/agricultural-crops-image-classification/Agricultural-crops/'
os.chdir(base_dir)

## 🗂️ Inspeção Inicial das Classes do Dataset

* Detecta todas as pastas no diretório base (cada uma representa uma classe).

* Conta quantas imagens existem em cada classe.

* Armazena os nomes das classes e a primeira imagem de cada uma pra uso posterior.

* Organiza tudo num DataFrame do pandas, ordenando pela quantidade de imagens por classe.

💡 Isso é útil pra identificar se o dataset está balanceado ou se há classes com muito mais imagens que outras (desequilíbrio de classes).

In [ ]:
# to list every directory name (label name)
directories_list = tf.io.gfile.listdir(base_dir)

# get number of labels
len_labels = len(directories_list)
print(f"Total Class Labels = {len_labels}")

vis_images = []; vis_labels =[]
length_file_list = []; label_list = []

for item in directories_list:

    # get each label directory
    item_dir = os.path.join(base_dir, item)
    # get list of images of each label
    item_files = os.listdir(item)
    # number of images per label
    len_per_label = len(os.listdir(item))

    length_file_list.append(len_per_label)
    label_list.append(item)

    # get first image of each label (for visualisation purpose)
    vis_images.append(os.path.join(item_dir, item_files[0]))
    # get respective label name (for visualisation purpose)
    vis_labels.append(item)

df_temp = pd.DataFrame({'Labels':label_list, 'Number of Images':length_file_list}).\
sort_values(by='Number of Images', ascending=False)
df_temp

 🖼️ Visualizando uma Imagem de Cada Classe

 Visualização das primeiras imagens do dataset, uma pra cada classe.

 * Cria uma figura com subplots organizados em grade para exibir uma imagem de cada classe.

* Usa as listas vis_images e vis_labels (criadas lá atrás) pra carregar a primeira imagem representativa de cada categoria.

* Remove ticks dos eixos, adiciona o rótulo de cada imagem e um título geral.

👀 Isso é ótimo pra ter uma visualização rápida da variedade de classes e garantir que os dados estão organizados corretamente.

🧠 Também ajuda a identificar possíveis problemas como rótulos trocados, imagens em branco, ou qualidade baixa de amostras.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(len(vis_labels)):
    plt.subplot(6,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    img = mpimg.imread(vis_images[i])
    plt.imshow(img)
    plt.xlabel(vis_labels[i])
    plt.suptitle(f"Classifying {len_labels} Types of Image Labels",fontsize=18, fontweight='bold')
plt.show()

## 🔀 Função para Dividir os Dados em Treino e Validação

 É um modo manual de dividir o dataset em treino e validação — e com uma vantagem extra: ela ignora arquivos corrompidos (com tamanho 0 bytes)e ignora eles! 🎯

* Usa random.sample pra sortear uma fração dos arquivos (SPLIT_SIZE) pro treino.

* Coloca o restante no conjunto de validação.

* Copia os arquivos para os diretórios respectivos: TRAINING_DIR e VALIDATION_DIR.

🔒 A semente aleatória (random.seed(42)) garante que a divisão seja reprodutível — ou seja, sempre vai gerar o mesmo resultado.

💡 Essa abordagem funciona bem quando o dataset já está separado por pastas de classe, e você quer gerar divisões manuais sem usar ImageDataGenerator.flow_from_directory() ainda.

In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

    selected_file_names = []
    all_file_names = os.listdir(SOURCE_DIR)
    for file_name in all_file_names:
        file_path = os.path.join(SOURCE_DIR, file_name)
        size = os.path.getsize(file_path)
        if size != 0:
              selected_file_names.append(file_name)
        else:
              print(f"{file_name} is zero length, so ignoring.")

    random.seed(42)
    selected_train_files = random.sample(selected_file_names, int(SPLIT_SIZE * len(selected_file_names)))
    selected_val_files = [x for x in selected_file_names if x not in selected_train_files]

    for file_name in selected_train_files:
        source = os.path.join(SOURCE_DIR, file_name)
        destination = os.path.join(TRAINING_DIR, file_name)
        copyfile(source, destination)

    for file_name in selected_val_files:
        source = os.path.join(SOURCE_DIR, file_name)
        destination = os.path.join(VALIDATION_DIR, file_name)
        copyfile(source, destination)

## 🗂️ Função para Criar Pastas de Treino e Validação Organizadas por Classe

Essa função é como o “modo automático” pra preparar as pastinhas de treino e validação pra cada classe. Ela organiza tudo bonitinho e ainda chama a função split_data().

* Percorre cada classe (cada pasta dentro do dataset).

* Cria pastas específicas pra treino e validação, organizadas assim:

    * root_path/_MODELLING/training/<classe>/
    * root_path/_MODELLING/validation/<classe>/

* Chama a função split_data() pra distribuir as imagens entre treino e validação, com o split_size definido (padrão = 90% treino, 10% validação).

🧹 Antes de rodar essa função, certifique-se de que os diretórios _MODELLING/training e validation não existem ainda, senão o os.makedirs() pode lançar erro.

💡 Muito útil pra deixar os dados prontos pra ImageDataGenerator.flow_from_directory(), que espera justamente essa estrutura por pastas.

In [ ]:
def create_train_val_dirs(root_path, split_size = 0.9):
    for item in directories_list:
        source_dir = os.path.join(base_dir, item)
        training_dir = os.path.join(root_path, f'_MODELLING/training/{item}')
        validation_dir = os.path.join(root_path, f'_MODELLING/validation/{item}')

        # Create EMPTY directory
        os.makedirs(training_dir)
        os.makedirs(validation_dir)

        split_data(source_dir, training_dir, validation_dir, split_size)
    print(f"Created training and validation directories containing images at split size of {split_size}")

## 🚧 Criando as Pastas com os Dados de Treino e Validação

* Executa a função create_train_val_dirs() que:

* Cria as pastas para treino e validação.

* Copia as imagens da pasta original (base_dir) para as novas pastas organizadas por classe.

* Divide os dados com 90% para treino e 10% para validação (split_size = 0.9).

📂 As imagens ficarão organizadas em
* /kaggle/working/_MODELLING/training/ e
* /kaggle/working/_MODELLING/validation/, prontos pra serem lidos por geradores de imagens.

⚠️ Essa célula pode demorar um pouco dependendo do tamanho do dataset, já que ela está copiando arquivos de forma individual.

In [ ]:
create_train_val_dirs('/kaggle/working', split_size = 0.9)

## 🖼️ Visualização de Data Augmentation com ImageDataGenerator

✨ Ela demonstra como o ImageDataGenerator transforma (aumenta) as imagens de forma automática, usando técnicas de data augmentation.🎨

* Carrega uma imagem de uma classe específica.

* Aplica quatro tipos diferentes de transformação usando o ImageDataGenerator:

* Rotação da imagem

* Deslocamento horizontal

* Zoom in/out

* Espelhamento horizontal

* Exibe 3 variações da imagem para cada tipo de transformação.

🔁 Isso ajuda o modelo a generalizar melhor, pois ele "vê" diferentes versões de uma mesma imagem durante o treino.

🧪 Ideal pra verificar se o augmentation está funcionando como esperado antes de aplicar no pipeline de treinamento.

In [ ]:
def show_ImageDataGenerator(vis_images, vis_labels, image_index):
    #Loads image in from the set image path
    class_label = vis_labels[image_index]
    img = tf.keras.preprocessing.image.load_img(vis_images[image_index], target_size= (250,250))
    img_tensor = tf.keras.preprocessing.image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)

    #Creates our batch of one image
    def show_image(datagen, param):
        pic = datagen.flow(img_tensor, batch_size =1)
        plt.figure(figsize=(10,3.5))
        #Plots our figures
        for i in range(1,4):
            plt.subplot(1, 3, i)
            batch = pic.next()
            image_ = batch[0].astype('uint8')
            plt.imshow(image_)
        plt.suptitle(f"Class: {class_label} \n Image Generator ({param})",fontsize=18, fontweight='bold')

        plt.show()

    datagen = ImageDataGenerator(rotation_range=30)
    show_image(datagen, "rotation_range=30")

    datagen = ImageDataGenerator(width_shift_range=0.2)
    show_image(datagen, "width_shift_range=0.2")

    datagen = ImageDataGenerator(zoom_range=0.2)
    show_image(datagen, "zoom_range=0.2")

    datagen = ImageDataGenerator(horizontal_flip=True)
    show_image(datagen, "horizontal_flip=True")

show_ImageDataGenerator(vis_images, vis_labels, image_index = 0)

## 🔄 Aplicando Data Augmentation em Outra Imagem

* Executa novamente a função show_ImageDataGenerator, agora usando a quarta imagem (índice 3) da lista vis_images.

* Exibe as transformações de data augmentation (rotação, deslocamento, zoom e espelhamento) aplicadas a essa nova imagem.

🎯 Isso serve pra explorar como o ImageDataGenerator afeta diferentes tipos de imagem no dataset — uma ótima prática pra garantir que as transformações estão mantendo a coerência visual das classes.

In [ ]:
show_ImageDataGenerator(vis_images, vis_labels, image_index = 3)

## 🔄 Função para Criar os Geradores de Imagem (Treino e Validação)

Aqui definimos uma função super importante: criar os geradores de imagem para treino e validação, com data augmentation no treino e normalização simples na validação.

* Cria dois ImageDataGenerator:

* Um com várias transformações (rotação, deslocamento, zoom, flip) para aumentar a variedade no conjunto de treino.

* Outro apenas com rescale=1./255 para normalizar as imagens de validação.

* Usa flow_from_directory() para gerar batches de imagens automaticamente a partir da estrutura de pastas:

    *<TRAINING_DIR>/<classe1>/
    *<TRAINING_DIR>/<classe2>/
...
* Retorna dois geradores que serão usados no model.fit() mais adiante.

⚠️ Atenção: class_mode='binary' só funciona se houver duas classes. Se tiver mais, é melhor usar class_mode='categorical'.

📐 target_size=(150, 150) define o tamanho para o qual todas as imagens serão redimensionadas.

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):

    # Instantiate the ImageDataGenerator class (don't forget to set the arguments to augment the images)
    train_datagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range=30,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

    # Pass in the appropriate arguments to the flow_from_directory method
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

    # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # Pass in the appropriate arguments to the flow_from_directory method
    validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode='binary',
                                                                target_size=(150, 150))
    return train_generator, validation_generator

## 📂 Definindo os Caminhos das Pastas de Treino e Validação

* Usa os.path.join pra montar os caminhos completos das pastas de treino e validação dentro da pasta _MODELLING.

* Imprime o caminho da pasta de validação para conferência.

🔎 Isso garante que as próximas funções, como os geradores de imagem, saibam exatamente onde buscar os dados.

✅ Se estiver rodando no Colab, substitua '/kaggle/working' por '/content'.

In [ ]:
training_dir = os.path.join('/kaggle/working', '_MODELLING', 'training')
validation_dir = os.path.join('/kaggle/working', '_MODELLING', 'validation')

print(validation_dir)

## 🧪 Gerando os Batches de Imagens para Treino e Validação

* Chama a função train_val_generators() com os diretórios de treino e validação definidos anteriormente.

* Cria dois objetos:

    * train_generator: que aplica data augmentation e fornece imagens em lotes para o treino.

    * validation_generator: que fornece imagens normalizadas para validação, sem transformação.

📦 Esses geradores são ideais para uso com model.fit() porque entregam os dados em tempo real (sem precisar carregar tudo na memória).

🧠 A partir daqui, já pode treinar modelos usando essas imagens em fluxo contínuo!

In [ ]:
train_generator, validation_generator = train_val_generators(training_dir, validation_dir)

## 🧠 Definindo o Modelo 1 – CNN Simples do Zero

Define o primeiro modelo de rede neural convolucional (CNN) do projeto — feito do zero, usando a API sequencial do Keras.

Define uma CNN simples, com:

* 2 camadas convolucionais + max pooling

* Uma camada Flatten para transformar a imagem em vetor

* Dropout de 50% para evitar overfitting

* Uma camada densa de 1024 neurônios

* Camada de saída softmax, com número de saídas igual ao total de classes (len_labels)

📏 A entrada é uma imagem RGB de 150x150 pixels (input_shape=(150, 150, 3)).

🧮 A função model_1.summary() exibe a arquitetura da rede com o total de parâmetros treináveis.

⚠️ Como é uma softmax, esse modelo assume classificação multiclasse com classes mutuamente exclusivas.

In [ ]:
model_1 = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(len_labels, activation='softmax')
])

# Print the model summary
model_1.summary()

## ⏹️ Callback para Parar o Treinamento com 80% de Acurácia

Define um callback personalizado — um tipo de “vigia” que monitora o treino e interrompe automaticamente quando a acurácia chega a 80%. Super útil pra evitar overfitting e poupar tempo de computação.

* Cria uma classe myCallback que herda de tf.keras.callbacks.Callback.

* No final de cada época (on_epoch_end), ela checa a acurácia (logs['accuracy']).

* Se a acurácia passar de 80%, ela:

* Imprime uma mensagem no console;

* Interrompe o treinamento com self.model.stop_training = True.

🧠 Esse tipo de callback é útil quando você quer evitar que o modelo “continue treinando à toa” depois de atingir um desempenho satisfatório.

⚠️ Funciona apenas se a métrica accuracy estiver sendo monitorada no model.compile().

In [ ]:
# Define a Callback class that stops training once accuracy reaches 80%
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.8):
            print("\nReached 80% accuracy so cancelling training!")
            self.model.stop_training = True
callbacks = myCallback()

## ⚙️ Compilando o Modelo 1 (CNN Simples)

Aqui é onde o modelo CNN é compilado, ou seja, configurado com otimizador, função de perda e métrica antes de ser treinado.

* Otimizador: Usa o Adam, um dos otimizadores mais eficientes e populares para deep learning.

* Taxa de aprendizado (learning_rate=0.001) define o “tamanho do passo” durante o ajuste dos pesos.

* Função de perda: sparse_categorical_crossentropy — ideal quando os rótulos das classes são números inteiros (ex: 0, 1, 2...) e não one-hot encoded.

* Métrica: accuracy, para acompanhar a porcentagem de classificações corretas durante o treino e validação.

🧠 Lembrando: sparse_categorical_crossentropy espera que os rótulos estejam em formato inteiro, diferente de categorical_crossentropy que exige one-hot (vetores binários).

In [ ]:
model_1.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
            loss = 'sparse_categorical_crossentropy',
            metrics=['accuracy'])

## 🚀 Treinando o Modelo 1

Agora iniciamos o treinamento do modelo usando os dados gerados pelas pastas e com o callback de parada automática ativado.

Treina o model_1 usando:

* train_generator: imagens com data augmentation.

* validation_generator: imagens normalizadas (sem transformação).

* Executa por até 20 épocas, mas pode parar antes se a acurácia de treino passar de 80%, graças ao callbacks=callbacks.

* Armazena o histórico do treinamento (loss, accuracy etc) no objeto history_1.

📈 Esse histórico pode ser usado depois pra gerar gráficos de desempenho com matplotlib.

⏱️ O tempo de execução depende do tamanho do dataset e do modelo. Com ImageDataGenerator, as imagens são carregadas e transformadas em tempo real (ótimo pra memória!).

In [ ]:
history_1 = model_1.fit(train_generator,
                    epochs=20,
                    validation_data=validation_generator,
                    callbacks=callbacks)

## 📊 Visualização da Performance do Modelo (Acurácia & Perda)

Nessa célula, não só plotamos os gráficos de treino e validação, como ainda calculamos o gradiente (m) da curva, o que é ótimo pra entender se o modelo está melhorando de forma consistente.

* Define uma função vis_evaluation() para:

* Plotar dois gráficos lado a lado:

* Acurácia de treino e validação

* Perda (loss) de treino e validação

* Calcular o gradiente da curva (m) — a taxa de variação da métrica ao longo das épocas (tipo: "o quanto melhorou por época").

* Exibir essas informações visualmente com legendas e anotações automáticas nos gráficos.

Depois:

* Extrai o histórico salvo durante o model.fit() com history_1.history

* Chama a função para visualizar os resultados do modelo 1 (CNN básica)

📈 Esse tipo de visualização ajuda a identificar overfitting (ex: acurácia de treino sobe, mas a de validação não) ou underfitting (quando ambas são ruins).

🧠 O gradiente ("m") mostra se o modelo está melhorando de forma consistente ou se estagnou.

In [ ]:
def vis_evaluation(history_dict, model_name):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))
    epochs = range(1, len(history_dict['accuracy'])+1)

    def get_gradient(y_arr, epochs):
        return round((y_arr[-1] - y_arr[0]) / (epochs[-1] - epochs[0]),2)

    def vis_sub_evaluation(n, Accuracy, train_acc, val_acc, epochs):
        axs[n].plot(epochs, train_acc, label=f'Training {Accuracy}', ls='--')
        axs[n].plot(epochs, val_acc, label=f'Validation {Accuracy}', ls='dotted')

        axs[n].set_title(f'Training and Validation {Accuracy}')
        axs[n].set_xlabel('Epochs')
        axs[n].set_ylabel(Accuracy)

        handles, labels = axs[n].get_legend_handles_labels()
        m_patch = mpatches.Patch(color='grey',label='m: gradient')
        handles.append(m_patch)
        axs[n].legend(handles=handles)

        def annotate_box(train_acc):
            return AnnotationBbox(TextArea(f"m = {get_gradient(train_acc, epochs)}"), (epochs[-1], train_acc[-1]),
                            xybox=(20, 20),
                            xycoords='data',
                            boxcoords="offset points",
                            arrowprops=dict(arrowstyle="->"))
        axs[n].add_artist(annotate_box(train_acc))
        axs[n].add_artist(annotate_box(val_acc))

    train_acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    vis_sub_evaluation(0, 'Accuracy', train_acc, val_acc, epochs)

    train_loss = history_dict['loss']
    val_loss = history_dict['val_loss']
    vis_sub_evaluation(1, 'Loss', train_loss, val_loss, epochs)

    plt.suptitle(f"Performance Evaluation of {model_name}",fontsize=18, fontweight='bold')
    plt.show()

history_dict_1 = history_1.history
vis_evaluation(history_dict_1, 'Basic CNN')

## 🚀 Carregando a Base do Modelo Pré-Treinado (VGG16)

Entramos no território do Transfer Learning com VGG16 — e essa célula prepara o modelo pré-treinado para ser usado como base na nossa rede.

* Carrega a arquitetura VGG16 pré-treinada no ImageNet, sem o topo (as camadas densas finais): include_top=False.

* Define que nenhuma camada do modelo será treinável (layer.trainable = False), ou seja, a rede será usada apenas como extratora de características visuais.

* Conta e imprime o número total de parâmetros e quantos são treináveis (neste caso, deve dar zero treináveis).

📦 Transfer Learning = usar o "conhecimento visual" de um modelo já treinado em um grande dataset (como ImageNet) e aplicar em outro problema.

🔒 Congelar as camadas impede que os pesos da VGG sejam alterados — ótimo pra economizar tempo e evitar overfitting com datasets pequenos.

In [ ]:
from tensorflow.keras.applications import VGG16

pre_trained_model = VGG16(include_top=False,weights='imagenet', input_shape=(150, 150, 3))
for layer in pre_trained_model.layers:
    layer.trainable = False

total_params = pre_trained_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in pre_trained_model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

## 🔍 Explorando a Saída do Modelo Pré-Treinado

Mostra como termina a VGG16 (a última camada convolucional) e confirma o tipo do objeto que foi carregado.

* Armazena a última saída do modelo VGG16 congelado na variável last_output.

* Essa saída é o que será usado como entrada para o "topo personalizado", que será adicionado na próxima etapa.

* Imprime o tipo do objeto carregado (tensorflow.keras.Model), confirmando que ele é um modelo funcional do Keras.

📐 A saída da VGG16 (sem o topo) é um tensor 3D com várias ativações (mapas de características) — isso alimentará as camadas densas que vêm a seguir.

🔌 Esse last_output será conectado via Functional API ao novo "topo" que você vai construir.

In [ ]:
last_output = pre_trained_model.output
print('last layer output: ', last_output)

# Print the type of the pre-trained model
print(f"The pretrained model has type: {type(pre_trained_model)}")

## 🧱 Construindo o Modelo Final com Transfer Learning

Função que monta o modelo completo de Transfer Learning, usando a VGG16 congelada como base e adicionando um topo personalizado.

* Achata (Flatten) a saída da VGG16 pra transformar os mapas de ativação num vetor 1D.

* Adiciona:

    * Uma camada densa com 1024 unidades e ReLU.

    * Um dropout de 30% pra evitar overfitting.

    * Uma camada final Dense com ativação softmax, com número de saídas igual ao total de classes (len_labels).

* Cria um modelo final com a entrada original da VGG e a nova saída personalizada.

⚙️ A função retorna o modelo completo, pronto pra compilar e treinar.

🧠 Isso é o “melhor dos dois mundos”: aproveita o poder da VGG pra extrair padrões visuais e usa um classificador próprio pra sua tarefa.

In [ ]:
from tensorflow.keras import Model

def transfer_learning(last_output, pre_trained_model):
    # Flatten da saída da VGG
    x = tf.keras.layers.Flatten()(last_output)
    # Camada densa com 1024 neurônios e ativação ReLU
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    # Dropout para reduzir overfitting
    x = tf.keras.layers.Dropout(0.3)(x)
    # Camada de saída com softmax para classificação multiclasse
    x = tf.keras.layers.Dense(len_labels, activation='softmax')(x)

    # Modelo final unindo entrada da VGG16 e novo topo
    model = Model(inputs=pre_trained_model.input, outputs=x)

    return model


## 🧬 Criando e Resumindo o Modelo com Transfer Learning (Modelo 2)

O  modelo 2 está oficialmente criado! Essa célula chama a função que definimos e imprime o resumo da arquitetura completa — VGG16 como base + seu topo customizado.

Chama a função transfer_learning() usando:

* last_output: a saída da VGG16 (camadas convolucionais).

* pre_trained_model: o modelo base congelado.

Gera um modelo completo (model_2) com a entrada da VGG16 e uma nova "cabeça" de classificação.

Exibe um resumo com:

* Quantidade de camadas

* Formato das saídas

* Total de parâmetros

* Quantos são treináveis (neste caso, só as camadas do topo)

📌 As camadas da VGG16 estão congeladas (não treináveis), então só o “topo” é atualizado durante o treinamento.

📈 Ideal pra quando você quer boas representações visuais sem precisar treinar uma CNN do zero — especialmente útil com datasets pequenos.

In [ ]:
model_2 = transfer_learning(last_output, pre_trained_model)
model_2.summary()

## ⚙️ Compilando o Modelo 2 (Transfer Learning com VGG16)

Agora o modelo 2 (com VGG16) está sendo preparado pro combate! ⚔️
Essa célula faz a mesma etapa de compilação que foi feita no modelo 1, só que agora aplicada ao modelo de Transfer Learning.

* Usa o otimizador Adam, com taxa de aprendizado de 0.001.

* Define a função de perda como sparse_categorical_crossentropy, já que os rótulos são inteiros.

* Monitora a acurácia como métrica principal.

✅ Mesmo padrão de compilação usado no modelo 1 — assim a comparação entre eles será justa.

🧠 Lembra que só as camadas do “topo” do modelo são treináveis aqui, então o treinamento deve ser mais rápido e menos propenso a overfitting.

In [ ]:
model_2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
            loss = 'sparse_categorical_crossentropy',
            metrics=['accuracy'])

## 🚀 Treinando o Modelo 2 (Transfer Learning com VGG16)

🧠🔥Essa célula dá o start oficial no treinamento do modelo 2, agora usando a arquitetura VGG16 como base com o seu topo customizado. E ainda com o callback de parada automática ativado!

* Inicia o treinamento do model_2, que usa a VGG16 como extratora de características e um topo denso customizado como classificador.

* Treina por até 20 épocas, mas pode parar antes automaticamente se atingir 80% de acurácia (graças ao callbacks).

* Usa os mesmos geradores train_generator e validation_generator definidos anteriormente.

* Armazena o histórico do treinamento no objeto history_2.

⚡ Como só o topo do modelo é treinável, esse modelo costuma treinar mais rápido e atingir boa performance com menos épocas.

📊 Esse histórico será usado na próxima etapa para comparar os dois modelos visualmente.

In [ ]:
history_2 = model_2.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 20,
                    callbacks=callbacks)

## 📊 Visualização da Performance do Modelo 2 (Transfer Learning)

Fechamos com chave de ouro o ciclo de treinamento do modelo 2 — ela visualiza o desempenho do modelo baseado em VGG16, do mesmo jeitinho que fez com o modelo 1 🌟

* Extrai o histórico do treinamento (history_2.history) e armazena no dicionário history_dict_2.

* Chama a função vis_evaluation() para gerar:

* Gráfico de acurácia (treino x validação)

* Gráfico de perda (loss) (treino x validação)

* Com anotações automáticas mostrando o gradiente de melhoria (“m”).

📈 Isso permite comparar visualmente a performance do modelo 2 com o modelo 1, e verificar qual teve melhor generalização, menor perda e estabilidade nas curvas.

🧠 Uma curva de validação mais estável e com menor perda geralmente indica menos overfitting — ponto forte dos modelos com Transfer Learning.

In [ ]:
history_dict_2 = history_2.history
vis_evaluation(history_dict_2, 'Transfer Learning')